<a href="https://colab.research.google.com/github/weizhiyuan62/weizhiyuan62/blob/main/gaussian_splatting_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D Gaussian splatting
https://repo-sam.inria.fr/fungraph/3d-gaussian-splatting/

This notebook is composed by Andreas Lewitzki.

I made this to experiment with processing video of coice, convert structure from motion and build a model for export to local computer for viewing.
As some packages and tools are compiled for CUDA support and from scratch it will take some time to bootstrap. (~35min)

Ofcourse you could take this and build a container image and deploy precompiled elsewhere.

I've had success with both T4 and A100 instances and both 4k and 1080p video, but beware that processing 4k images takes considarable longer. Also, you wont get 4x point detail. ;)<br>
But that might also to some extent depend on features and scene.

Please note that the Structure-from-Motion process assumes at least some consistent objects. That means you probably will have better luck circling an object than just running around fps doom-style. Also, as we use a video file the resulting images will lack exif metadata that it othervise could pick up hints from.



#Install requirements and clone graphdecos gaussian-splatting project

Required dev environment for ceres-resolver and colmap

In [24]:

!sudo apt-get install \
    git \
    cmake \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libcgal-qt5-dev \
    libmetis-dev \
    libflann-dev \
    libatlas-base-dev \
    libsuitesparse-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-filesystem-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-regex-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libatlas-base-dev is already the newest version (3.10.3-12ubuntu1).
libboost-graph-dev is already the newest version (1.74.0.3ubuntu7).
libboost-test-dev is already the newest version (1.74.0.3ubuntu7).
libcgal-dev is already the newest version (5.4-1).
libcgal-qt5-dev is already the newest version (5.4-1).
libeigen3-dev is already the newest version (3.4.0-2ubuntu2).
libflann-dev is already the newest version (1.9.1+dfsg-11).
libgflags-dev is already the newest version (2.2.2-2).
libglew-dev is already the newest version (2.2.0-4).
libgoogle-glog-

 Ceres Solver 1 is an open source C++ library for modeling and solving large, complicated optimization problems. It can be used to solve Non-linear Least Squares problems with bounds constraints and general unconstrained optimization problems.

In [26]:


!git clone https://ceres-solver.googlesource.com/ceres-solver
%cd ceres-solver
!git checkout $(git describe --tags) # Checkout the latest release
%mkdir build
%cd build
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF -DCMAKE_CUDA_ARCHITECTURES=native
!make
!sudo make install

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,204 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,094 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,519 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jam

COLMAP is a general-purpose Structure-from-Motion (SfM) and Multi-View Stereo (MVS) pipeline with a graphical and command-line interface. It offers a wide range of features for reconstruction of ordered and unordered image collections.

In [13]:

!sudo apt update
!sudo apt install libabsl-dev
%cd ..
!git clone https://github.com/colmap/colmap
%cd colmap
!git checkout dev
%mkdir build
%cd build
!cmake .. -DCMAKE_CUDA_ARCHITECTURES=native
!make
!sudo make install
!CC=/usr/bin/gcc-6 CXX=/usr/bin/g++-6 cmake ..

/content/gaussian-splatting/ceres-solver/build/ceres-solver
Cloning into 'colmap'...
remote: Enumerating objects: 29108, done.
remote: Counting objects: 100% (1154/1154), done.
remote: Compressing objects: 100% (664/664), done.
remote: Total 29108 (delta 919), reused 496 (delta 490), pack-reused 27954 (from 3)
Receiving objects: 100% (29108/29108), 71.83 MiB | 30.02 MiB/s, done.
Resolving deltas: 100% (22265/22265), done.
/content/gaussian-splatting/ceres-solver/build/ceres-solver/colmap
error: pathspec 'dev' did not match any file(s) known to git
/content/gaussian-splatting/ceres-solver/build/ceres-solver/colmap/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-

clone graphdecos gaussian-splatting repo

In [14]:
%cd /content
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting
!pip install -q plyfile

# borrow some precompiled python libs from camenduru
%cd /content/gaussian-splatting
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.whl
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/simple_knn-0.0.0-cp310-cp310-linux_x86_64.whl



/content
fatal: destination path 'gaussian-splatting' already exists and is not an empty directory.
/content/gaussian-splatting
ERROR: diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.
ERROR: simple_knn-0.0.0-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.


# Setup project
Mount google drive and grab your video file and splitt up depending on fps. So if you've recorded @ 30fps the output will be a png image representing every 15th frame.

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
project_name = 'my3dgsproject1' # @param {type:"string"}
video_file = '/content/gdrive/MyDrive/test.MOV' # @param {type:"string"}
fps = 2 # @param {type:"slider", min:0, max:15, step:0.25}

!mkdir -p /content/{project_name}/input
!ffmpeg -i {video_file} -qscale:v 1 -qmin 1 -vf fps={fps} /content/{project_name}/input/%04d.jpg

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# Process images and train model
This is essentially a wrapper over colmap feature extract, matcher, mapper and undistorter. <br>
After this we train model and upload to gdrive.

In [17]:
%cd /content/gaussian-splatting
!python convert.py -s /content/{project_name}/

/content/gaussian-splatting
sh: 1: colmap: not found
ERROR:root:Feature extraction failed with code 32512. Exiting.


Train the model.
<b>-r 1</b> here means no resizing.

In [18]:
!python train.py -s /content/{project_name} -r 1 --model_path=./output/{project_name}
!cp -R /content/gaussian-splatting/output/{project_name} /content/gdrive/MyDrive/

Traceback (most recent call last):
  File "/content/gaussian-splatting/train.py", line 16, in <module>
    from gaussian_renderer import render, network_gui
  File "/content/gaussian-splatting/gaussian_renderer/__init__.py", line 14, in <module>
    from diff_gaussian_rasterization import GaussianRasterizationSettings, GaussianRasterizer
ModuleNotFoundError: No module named 'diff_gaussian_rasterization'
cp: cannot stat '/content/gaussian-splatting/output/my3dgsproject1': No such file or directory


# Now what?!
If you would like to just test the model localy, in my case windows, you may want to download:<br>
https://huggingface.co/camenduru/gaussian-splatting/resolve/main/GaussianViewTest.zip
<br><br>
If you have your google drive mounted in windows just unzip gaussianview then run something like:<br>
<i>GaussianViewTest\viewers\bin> .\SIBR_gaussianViewer_app.exe --model-path "gdrive\example\project_name"</i>

<br><br>
Bellow stuff is just for reference and optional.

In [19]:
!python render.py -m /content/gaussian-splatting/output/{project_name}

Traceback (most recent call last):
  File "/content/gaussian-splatting/render.py", line 13, in <module>
    from scene import Scene
  File "/content/gaussian-splatting/scene/__init__.py", line 16, in <module>
    from scene.dataset_readers import sceneLoadTypeCallbacks
  File "/content/gaussian-splatting/scene/dataset_readers.py", line 24, in <module>
    from scene.gaussian_model import BasicPointCloud
  File "/content/gaussian-splatting/scene/gaussian_model.py", line 21, in <module>
    from simple_knn._C import distCUDA2
ModuleNotFoundError: No module named 'simple_knn'


In [20]:
!ffmpeg -framerate 3 -i /content/gaussian-splatting/output/{project_name}/train/ours_30000/renders/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /renders.mp4
!ffmpeg -framerate 3 -i /content/gaussian-splatting/output/{project_name}/train/ours_30000/gt/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /gt.mp4 -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [21]:
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

!python train.py -s /content/gaussian-splatting/tandt/train

--2025-07-31 09:09:35--  https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.49, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/69/11/6911555bcb5edae447d514169831329398b45a3ed9ae3b00817878bc501559a1/816e62f22a161abbfe841d2a6b10cdf036e297c9fa289b3bfeee9c6ec526d7e1?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tandt_db.zip%3B+filename%3D%22tandt_db.zip%22%3B&response-content-type=application%2Fzip&Expires=1753956575&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1Mzk1NjU3NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82OS8xMS82OTExNTU1YmNiNWVkYWU0NDdkNTE0MTY5ODMxMzI5Mzk4YjQ1YTNlZDlhZTNiMDA4MTc4NzhiYzUwMTU1OWExLzgxNmU2MmYyMmExNjFhYmJmZTg0MWQyYTZiMTBjZGYwMzZlMjk3YzlmYTI4OWIzYmZlZWU5YzZlYzUyNmQ

In [22]:
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/GaussianViewTest.zip
!unzip GaussianViewTest.zip



--2025-07-31 09:09:51--  https://huggingface.co/camenduru/gaussian-splatting/resolve/main/GaussianViewTest.zip
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.49, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/69/11/6911555bcb5edae447d514169831329398b45a3ed9ae3b00817878bc501559a1/04543c347c164deaec3f0f4b585a7e9bb15b08a01607fdb4bfda88e8109d9238?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27GaussianViewTest.zip%3B+filename%3D%22GaussianViewTest.zip%22%3B&response-content-type=application%2Fzip&Expires=1753956591&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1Mzk1NjU5MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82OS8xMS82OTExNTU1YmNiNWVkYWU0NDdkNTE0MTY5ODMxMzI5Mzk4YjQ1YTNlZDlhZTNiMDA4MTc4NzhiYzUwMTU1OWExLzA0NTQzYzM0N2MxNjRkZWFlYzNmMGY0YjU4NWE3ZTliYjE1YjA4YTAxNjA